572 hw1 - Brad Bailey

Setup

In [2]:
import torch
import torchvision as tv
import numpy as np

Get the Data

In [ ]:
cifar10_dataset = tv.datasets.CIFAR10(root='./', # here
                               train=True, # train split
                               download=True, # we want to get the data
                               transform=tv.transforms.ToTensor(), # put it into tensor format
                          )
train_data = torch.utils.data.DataLoader(cifar10_dataset,
                        batch_size=2,
                        )

Investigation

In [ ]:
data = iter(train_data) # Let's iterate on it
single_point = next(data)
print(f"""Type: {type(single_point)}
Length: {len(single_point)}
More Types: {type(single_point[0])}, {type(single_point[1])}
Shapes: {single_point[0].shape}, {single_point[1].shape}
Labels: {single_point[1]}
""")

In [ ]:
import matplotlib.pyplot as plt
ToPIL = tv.transforms.ToPILImage() # Converting function
img0 = ToPIL(single_point[0][0])
img1 = ToPIL(single_point[0][1])
# Plotting
fig, axs = plt.subplots(1,2)
axs[0].imshow(img0)
axs[1].imshow(img1)

Linear Classifier

In [ ]:
class C10Linear(torch.nn.Module):
    def __init__(self,
                 ninputs=3*32*32,
                 nclasses=10
                ):
        super().__init__()
        self.linear = torch.nn.Linear(ninputs, nclasses)
        
    def forward(self, x):
        x = x.reshape(-1, 3072)
        x = self.linear(x)
        return x

Training

In [ ]:
from torch import optim
from tqdm import tqdm # This is optional but useful

# Let's get the right torch device (preference of GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Let's set up some parameters
learning_rate=5e-1
nepochs = 10
ninputs=3*32*32
nclasses=10

model = C10Linear(ninputs=ninputs,
                  nclasses=nclasses).to(device)
print(model)
# We need an optimizer that tells us what form of gradient descent to do
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# We also need a loss function
LossFunction = torch.nn.CrossEntropyLoss()

# We're going to reload the data here so we have added clarity

batch_size = 100

cifar10_dataset = tv.datasets.CIFAR10(root='./', # here
                               train=True, # train split
                               download=True, # we want to get the data
                               transform=tv.transforms.ToTensor(), # put it into tensor format
                        )
train_data = torch.utils.data.DataLoader(cifar10_dataset,
                        batch_size=batch_size,
                        )

In [ ]:
model.train()
loss_history = []
loss = torch.Tensor([0])
for epoch in tqdm(range(nepochs),
                  desc=f"Epoch",
                  unit="epoch",
                  disable=False):
    for (data, label) in tqdm(train_data,
                              desc="iteration",
                              unit="%",
                              disable=True):
        optimizer.zero_grad(set_to_none=True) # Here we clear the gradients
        
        # We need to make sure the tensors are on the same device as our model
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        
        loss = LossFunction(out, label)
        
        # PyTorch is Magic!
        loss.backward() # This function calculates all our gradients
        optimizer.step() # This function does our gradient descent with those gradients
        loss_history.append(loss.item())
    print(f"Epoch {epoch}: loss: {loss.item()}")

In [ ]:
# Note that we are not plotting loss per epoch but per iteration
plt.plot(loss_history)
plt.title("Neural Network Loss")
plt.xlabel("Number of iterations")
plt.ylabel("Loss")
plt.show()

Testing testing

In [ ]:
cifar10_dataset = tv.datasets.CIFAR10(root='./', # here
                               train=True, # train split
                               download=True, # we want to get the data
                               transform=tv.transforms.ToTensor(), # put it into tensor format
                        )
test_data = torch.utils.data.DataLoader(cifar10_dataset,
                        batch_size=batch_size,
                        )

model.eval()
accuracy = 0
for (data, label) in test_data:
    data = data.to(device)
    label = label.to(device)
    out = model(data)
    answers = out.max(dim=1)[1]
    accuracy += (answers == label).sum()
print(f"Total accuracy = {accuracy / len(cifar10_dataset)*100:.2f}%")

CNN from scratch

In [ ]:
class scratchNet(torch.nn.Module):
    def __init__(self,
                 ninputs=3*32*32,
                 nclasses=10
                ):
        super().__init__()
        self.net = torch.nn.Sequential(
            
        torch.nn.Conv2d(3, 32, kernel_size=3, padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 2), #64*16*16
            
        torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 2), #128*8*8
            
        torch.nn.Flatten(),
        torch.nn.Linear(128*8*8, 1024), #correct dimensions???
        torch.nn.Linear(1024, 512),
        torch.nn.Linear(512, 10)
        )
        
    def forward(self, x):
        return self.net(x)

Training

In [ ]:
from torch import optim
from tqdm import tqdm # This is optional but useful

# Let's get the right torch device (preference of GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Let's set up some parameters
learning_rate=1
nepochs = 10
ninputs=3*32*32
nclasses=10

model = scratchNet(ninputs=ninputs,
                  nclasses=nclasses).to(device)
print(model)
# We need an optimizer that tells us what form of gradient descent to do
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# We also need a loss function
LossFunction = torch.nn.CrossEntropyLoss()

# We're going to reload the data here so we have added clarity

batch_size = 100

cifar10_dataset = tv.datasets.CIFAR10(root='./', # here
                               train=True, # train split
                               download=True, # we want to get the data
                               transform=tv.transforms.ToTensor(), # put it into tensor format
                        )
train_data = torch.utils.data.DataLoader(cifar10_dataset,
                        batch_size=batch_size,
                        )

In [ ]:
model.train()
loss_history = []
loss = torch.Tensor([0])
for epoch in tqdm(range(nepochs),
                  desc=f"Epoch",
                  unit="epoch",
                  disable=False):
    for (data, label) in tqdm(train_data,
                              desc="iteration",
                              unit="%",
                              disable=True):
        optimizer.zero_grad(set_to_none=True) # Here we clear the gradients
        
        # We need to make sure the tensors are on the same device as our model
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        
        loss = LossFunction(out, label)
        
        # PyTorch is Magic!
        loss.backward() # This function calculates all our gradients
        optimizer.step() # This function does our gradient descent with those gradients
        loss_history.append(loss.item())
    print(f"Epoch {epoch}: loss: {loss.item()}")

In [ ]:
# Note that we are not plotting loss per epoch but per iteration
plt.plot(loss_history)
plt.title("Neural Network Loss")
plt.xlabel("Number of iterations")
plt.ylabel("Loss")
plt.show()

In [ ]:
cifar10_dataset = tv.datasets.CIFAR10(root='./', # here
                               train=True, # train split
                               download=True, # we want to get the data
                               transform=tv.transforms.ToTensor(), # put it into tensor format
                        )
test_data = torch.utils.data.DataLoader(cifar10_dataset,
                        batch_size=batch_size,
                        )

model.eval()
accuracy = 0
for (data, label) in test_data:
    data = data.to(device)
    label = label.to(device)
    out = model(data)
    answers = out.max(dim=1)[1]
    accuracy += (answers == label).sum()
print(f"Total accuracy = {accuracy / len(cifar10_dataset)*100:.2f}%")